# <center>Data Mining project: Discover and describe areas of interest<br> and events from geo-located parsed_data</center>

## #0: Import Dataset and Libraries

## Setting Environment

In [1]:
! python -m venv dataMiningEnv
#Activate windows
#! dataMiningEnv\Scripts\activate.bat

#Activate mac
! source dataMiningEnv/bin/activate


## Importing Libraries

- ### Installs

In [52]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3
# Language processing
! pip install nltk
! python -m nltk.downloader popular # popular functions
# Folium
! pip install folium==0.12.1


You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgradin

- ### Imports

In [53]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
import sklearn.cluster as cl
import folium
from folium.plugins import MarkerCluster
import nltk


## Importing Data

In [54]:
# Charger les données avec low_memory=False pour éviter les avertissements
data = pd.read_csv("data/flickr_data2.csv", sep=",", low_memory=False)
data.head(5)

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN


## #1: Discovering areas of interests using clustering

## Understanding the Data

--Write something to describe this part of the report--

## Data clearing and preparation

In [55]:
# Étape 1 : Nettoyage initial des colonnes
# Supprimer les espaces supplémentaires dans les noms des colonnes
parsed_data = data
parsed_data.columns = parsed_data.columns.str.strip()

# Convertir les colonnes temporelles en numériques
# Liste des colonnes temporelles
time_columns = [
    'date_taken_minute', 'date_taken_hour', 'date_taken_day',
    'date_taken_month', 'date_taken_year',
    'date_upload_minute', 'date_upload_hour', 'date_upload_day',
    'date_upload_month', 'date_upload_year'
]

# Convertir chaque colonne en int64, remplacer les erreurs par 0
for col in time_columns:
    parsed_data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype('int64')

In [56]:
# Étape 2 : Suppression des doublons basés sur l'identifiant unique
# Sauvegarder les doublons pour audit futur
parsed_data[parsed_data['id'].duplicated(keep='first')].sort_values("id").to_csv(
    "data/parsed_lines/duplicatedId.csv", index=False
)

# Supprimer les doublons
parsed_data = parsed_data[~parsed_data['id'].duplicated(keep='first')]

In [57]:
# Étape 3 : Gestion des colonnes inutilisées ou corrompues
# Identifier les colonnes inutiles
unused_columns = ["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", 
                  "date_upload_minute", "date_upload_hour", "date_upload_day", 
                  "date_upload_month", "date_upload_year"]

# Sauvegarder les données corrompues
parsed_data[parsed_data[unused_columns].notnull().any(axis=1)].to_csv(
    "data/parsed_lines/corrupted_data.csv"
, index=False)

# Supprimer les colonnes inutilisées et les lignes corrompues
parsed_data = parsed_data.drop(columns=unused_columns)

In [58]:
# Étape 4 : Nettoyage des coordonnées GPS
# Définir les limites géographiques de Lyon
lyon_lat_min, lyon_lat_max = 45.69, 45.85
lyon_lon_min, lyon_lon_max = 4.78, 4.92

# Filtrer les données pour garder uniquement les points dans Lyon
parsed_data = parsed_data[
    (parsed_data['lat'] >= lyon_lat_min) & 
    (parsed_data['lat'] <= lyon_lat_max) &
    (parsed_data['long'] >= lyon_lon_min) &
    (parsed_data['long'] <= lyon_lon_max)
]

In [59]:
# Étape 5 : Gestion des valeurs manquantes
# Supprimer les lignes avec des valeurs manquantes dans latitude/longitude
parsed_data = parsed_data.dropna(subset=['lat', 'long'])

# Remplir les valeurs manquantes pour les colonnes textuelles par des chaînes vides
parsed_data['tags'] = parsed_data['tags'].fillna('')
parsed_data['title'] = parsed_data['title'].fillna('')

In [ ]:
# Étape 6 : Suppression des doublons géographiques et temporels
# Identifier les doublons basés sur latitude, longitude, et date
# --> PASSER ÇA EN POST-PROCESSING


In [60]:
# Étape 7 : Filtrage des dates incohérentes
# Garder uniquement les dates raisonnables (entre 2009 et 2025)
parsed_data = parsed_data[
    (parsed_data['date_taken_year'] >= 2009) & (parsed_data['date_taken_year'] <= 2025) &
    (parsed_data['date_taken_month'] >= 1) & (parsed_data['date_taken_month'] <= 12) &
    (parsed_data['date_taken_day'] >= 1) & (parsed_data['date_taken_day'] <= 31) &
    (parsed_data['date_taken_hour'] >= 0) & (parsed_data['date_taken_hour'] <= 23) &
    (parsed_data['date_taken_minute'] >= 0) & (parsed_data['date_taken_minute'] <= 59) 
]


In [61]:
# Étape 8 : Nettoyage des colonnes textuelles
# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux et passer en minuscules
    return re.sub(r'[^a-zA-Z0-9, ]', '', text).lower()

# Appliquer le nettoyage sur les colonnes textuelles
parsed_data['tags'] = parsed_data['tags'].apply(clean_text)
parsed_data['title'] = parsed_data['title'].apply(clean_text)

In [62]:
# Étape 9 : Normalisation des coordonnées GPS
# Normaliser latitude et longitude pour les algorithmes de clustering
#scaler = StandardScaler()
#parsed_data[['lat', 'long']] = scaler.fit_transform(parsed_data[['lat', 'long']])

In [63]:
# Résumé final des données nettoyées
print("Données nettoyées :")
print(parsed_data.info())

parsed_data.head(5)

Données nettoyées :
<class 'pandas.core.frame.DataFrame'>
Index: 155520 entries, 0 to 419137
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 155520 non-null  int64  
 1   user               155520 non-null  object 
 2   lat                155520 non-null  float64
 3   long               155520 non-null  float64
 4   tags               155520 non-null  object 
 5   title              155520 non-null  object 
 6   date_taken_minute  155520 non-null  int64  
 7   date_taken_hour    155520 non-null  int64  
 8   date_taken_day     155520 non-null  int64  
 9   date_taken_month   155520 non-null  int64  
 10  date_taken_year    155520 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 14.2+ MB
None


,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhne,chaise,rhnealpes",chaises avec vue,11,15,28,2,2010
1,4394748717,35853470@N00,45.753270,4.862953,,,51,17,28,2,2010
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59365 r46 v103 b163,29,17,28,2,2010
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",20100129 toiou avott lyon,15,20,28,1,2010
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",20100128 toiou avott lyon,10,20,28,1,2010


In [64]:
# Sauvegarder les données nettoyées pour les prochaines étapes
parsed_data.to_csv("data/cleaned_flickr_data.csv", index=False)

## Visualize geolocated points on a map

In [65]:
# Define the center of Lyon
lyon_lat, lyon_lon = 45.75, 4.85  # Approximate coordinates for Lyon, France

# Create a Folium map
map_lyon = folium.Map(location=[lyon_lat, lyon_lon], zoom_start=12)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(map_lyon)

# Add points from the dataset
for _, row in parsed_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],  # Replace with 'lat' and 'long' if necessary
        popup=f"Tags: {row['tags']}, Title: {row['title']}",  # Optional popup information
    ).add_to(marker_cluster)

# Display the map
map_lyon

# Optional: Save the map as an HTML file
map_lyon.save("lyon_map.html")
